# 2. Data Preprocssing

In [86]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [87]:
mov_fast = pd.read_csv('../data/processed/movement_fast_stat.csv')
mov_slow = pd.read_csv('../data/processed/movement_slow_stat.csv')
traffic_fast = pd.read_csv('../data/processed/traffic_fast_stat.csv')
traffic_slow = pd.read_csv('../data/processed/traffic_slow_stat.csv')

## 2.1. Fix Features Naming

In [88]:
# Removing the empty spaces before and after the featurs names
mov_fast.columns = mov_fast.columns.str.strip()
mov_slow.columns = mov_slow.columns.str.strip()
traffic_fast.columns = traffic_fast.columns.str.strip()
traffic_slow.columns = traffic_slow.columns.str.strip()

In [89]:
mov_fast.rename(columns={"ID_":"ID", "time_interval_":"time_interval"}, inplace=True)
mov_slow.rename(columns={"ID_":"ID", "time_interval_":"time_interval"}, inplace=True)

## 2.2. Columns Variability 

In [90]:
def find_non_varying_variables(df):
    non_varying_columns = []
    variability_percentage = []
    
    for column in df.columns:
        unique_count = df[column].nunique()
        total_count = len(df[column])
        variability = unique_count / total_count * 100
        
        if unique_count == 1:
            non_varying_columns.append(column)
            variability_percentage.append(variability)
    
    result_df = pd.DataFrame({'Variable': non_varying_columns, 'Variability Percentage': variability_percentage})
    return result_df

In [91]:
find_non_varying_variables(mov_fast)

,Variable,Variability Percentage
0,TouchButtons_min,0.027778


In [92]:
find_non_varying_variables(mov_slow)

,Variable,Variability Percentage


In [93]:
find_non_varying_variables(traffic_fast)

,Variable,Variability Percentage
0,size_min,0.027778
1,size_25%,0.027778
2,size_50%,0.027778
3,size_75%,0.027778


In [94]:
find_non_varying_variables(traffic_slow)

,Variable,Variability Percentage
0,size_min,0.027785
1,size_25%,0.027785
2,size_75%,0.027785


for the movement data, the data acquired from sensor 0, 1, 2, touchbutton,and remotebuttons are constants which will have to drop from our dataset, to focus only on meangful variables, that can help us make better predicition, for the traffic data, the only constant data are the ones related to packets size, it's important first to understand why those features are constans accross all the participants/games

In [95]:
mov_fast.drop(columns=find_non_varying_variables(mov_fast)['Variable'], inplace=True)
mov_slow.drop(columns=find_non_varying_variables(mov_slow)['Variable'], inplace=True)
traffic_fast.drop(columns=find_non_varying_variables(traffic_fast)['Variable'], inplace=True)
traffic_slow.drop(columns=find_non_varying_variables(traffic_slow)['Variable'], inplace=True)

## 2.3. Missing Values

In [96]:
def missing_columns(dataframe):
    """
    Returns a dataframe that contains missing column names and 
    percent of missing values in relation to the whole dataframe.
    
    dataframe: dataframe that gives the column names and their % of missing values
    """
    
    # find the missing values
    missing_values = dataframe.isnull().sum().sort_values(ascending=False)
    
    # percentage of missing values in relation to the overall size
    missing_values_pct = 100 * missing_values/len(dataframe)
    
    # create a new dataframe which is a concatinated version
    concat_values = pd.concat([missing_values, missing_values/len(dataframe),missing_values_pct.round(1)],axis=1)

    # give new col names
    concat_values.columns = ['Missing Count','Missing Count Ratio','Missing Count %']
    
    # return the required values
    return concat_values[concat_values.iloc[:,1]!=0]

In [97]:
missing_columns(mov_fast)

,Missing Count,Missing Count Ratio,Missing Count %


In [98]:
missing_columns(mov_slow)

,Missing Count,Missing Count Ratio,Missing Count %


In [99]:
missing_columns(traffic_fast)

,Missing Count,Missing Count Ratio,Missing Count %


In [100]:
missing_columns(traffic_slow)

,Missing Count,Missing Count Ratio,Missing Count %


## 2.4. Feature Scaling

In [101]:
def scaling(df):
    # Select numeric columns only
    numeric_cols = df.select_dtypes(include=['number']).columns.difference(["time_interval"])
    
    # Apply MinMaxScaler to the numeric columns
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[numeric_cols])
    
    # Create a DataFrame from the scaled data
    scaled_df = pd.DataFrame(scaled_data, columns=numeric_cols)
    
    # Re-include the string columns back into the DataFrame
    for col in df.columns:
        if col not in numeric_cols:
            scaled_df[col] = df[col].values
            
    return scaled_df

In [102]:
mov_fast = scaling(mov_fast)
mov_slow = scaling(mov_slow)
traffic_fast = scaling(traffic_fast)
traffic_slow = scaling(traffic_slow)

## 2.5. Label Encoding

In [103]:

def encoding(df):
    le = LabelEncoder()

    # iterate through all the categorical columns
    for col in df.select_dtypes('object').columns.difference(["ID"]):
        df[col] = le.fit_transform(df[col].astype(str))
    return df

In [104]:
mov_fast = encoding(mov_fast)
mov_slow = encoding(mov_slow)
traffic_fast = encoding(traffic_fast)
traffic_slow = encoding(traffic_slow)

## 2.6. Matching Columns
The fast movement dataset and slow movement dataset should have the same number of columns and the same exact features, as we are going to train on the slow movement dataset and predict on the fast movement

In [105]:
print('mov_slow Features shape: ', mov_slow.shape)
print('mov_fast Features shape: ', mov_fast.shape)
print('traffic_slow Features shape: ', traffic_slow.shape)
print('traffic_fast Features shape: ', traffic_fast.shape)

mov_slow Features shape:  (3597, 614)
mov_fast Features shape:  (3600, 613)
traffic_slow Features shape:  (3599, 38)
traffic_fast Features shape:  (3600, 37)


In [106]:
def match_columns(training_set,testing_set):
    """Matches the count of columns from training set to testing set by adding extra cols and setting them to 0."""
    
    for column in training_set.columns:
        if column not in testing_set.columns:
            testing_set[column]=0
    for column in testing_set.columns:
        if column not in training_set.columns:
            testing_set = testing_set.drop(column)
    return testing_set 

In [107]:
mov_fast = match_columns(mov_slow, mov_fast)
traffic_fast = match_columns(traffic_slow, traffic_fast)

In [108]:
print('mov_slow Features shape: ', mov_slow.shape)
print('mov_fast Features shape: ', mov_fast.shape)
print('traffic_slow Features shape: ', traffic_slow.shape)
print('traffic_fast Features shape: ', traffic_fast.shape)

mov_slow Features shape:  (3597, 614)
mov_fast Features shape:  (3600, 614)
traffic_slow Features shape:  (3599, 38)
traffic_fast Features shape:  (3600, 38)


## 2.7 Preparing Labels

In [109]:

IDs = {'group1_order1_user0': 1, 'group1_order1_user1': 2, 'group1_order1_user10': 3, 'group1_order1_user11': 4, 'group1_order1_user12': 5, 
       'group1_order1_user13': 6, 'group1_order1_user14': 7, 'group1_order1_user2': 8, 'group1_order1_user3': 9, 'group1_order1_user4': 10,
       'group1_order1_user5': 11, 'group1_order1_user6': 12, 'group1_order1_user7': 13, 'group1_order1_user8': 14, 'group1_order1_user9': 15, 
       'group1_order2_user0': 16, 'group1_order2_user1': 17, 'group1_order2_user10': 18, 'group1_order2_user11': 19, 'group1_order2_user12': 20, 
       'group1_order2_user13': 21, 'group1_order2_user14': 22, 'group1_order2_user2': 23, 'group1_order2_user3': 24, 'group1_order2_user4': 25, 
       'group1_order2_user5': 26, 'group1_order2_user6': 27, 'group1_order2_user7': 28, 'group1_order2_user8': 29, 'group1_order2_user9': 30, 
       'group2_order1_user0': 31, 'group2_order1_user1': 32, 'group2_order1_user10': 33, 'group2_order1_user11': 34, 'group2_order1_user12': 35, 
       'group2_order1_user13': 36, 'group2_order1_user14': 37, 'group2_order1_user2': 38, 'group2_order1_user3': 39, 'group2_order1_user4': 40, 
       'group2_order1_user5': 41, 'group2_order1_user6': 42, 'group2_order1_user7': 43, 'group2_order1_user8': 44, 'group2_order1_user9': 45, 
       'group2_order2_user0': 46, 'group2_order2_user1': 47, 'group2_order2_user10': 48, 'group2_order2_user11': 49, 'group2_order2_user12': 50, 
       'group2_order2_user13': 51, 'group2_order2_user14': 52, 'group2_order2_user2': 53, 'group2_order2_user3': 54, 'group2_order2_user4': 55, 
       'group2_order2_user5': 56, 'group2_order2_user6': 57, 'group2_order2_user7': 58, 'group2_order2_user8': 59, 'group2_order2_user9': 60}

def prepare_label(df, IDs):
    df['ID'] = df['ID'].apply(lambda x : x.split('/')[4])
    df['ID'] = df['ID'].apply(lambda x: IDs[x] - 1 )
    return df

In [110]:
mov_fast = prepare_label(mov_fast, IDs)
mov_slow = prepare_label(mov_slow, IDs)
traffic_fast = prepare_label(traffic_fast, IDs)
traffic_slow = prepare_label(traffic_slow, IDs)

In [111]:
mov_slow = mov_slow.reindex(sorted(mov_slow.columns), axis=1)
mov_fast = mov_fast.reindex(sorted(mov_fast.columns), axis=1)
traffic_slow = traffic_slow.reindex(sorted(traffic_slow.columns), axis=1)
traffic_fast = traffic_fast.reindex(sorted(traffic_fast.columns), axis=1)

In [112]:
mov_fast.to_csv('../data/processed/movement_fast_stat_cleaned.csv')
mov_slow.to_csv('../data/processed/movement_slow_stat_cleaned.csv')
traffic_fast.to_csv('../data/processed/traffic_fast_stat_cleaned.csv')
traffic_slow.to_csv('../data/processed/traffic_slow_stat_cleaned.csv')